In [7]:
import mysql.connector
import pandas as pd
import time
import datetime

In [8]:
cnx = mysql.connector.connect(user='openupadmin', password='6C7E2F760ABFF371',
                              host='openupdb.cxzofl0nts6g.us-east-1.rds.amazonaws.com', port = 3306,
                              database='Openup_db')

In [9]:
cursor = cnx.cursor()

In [10]:
cursor.execute(query)

NameError: name 'query' is not defined

In [11]:
data = pd.DataFrame(cursor.fetchall())

InterfaceError: No result set to fetch from.

# All user purchase data, return cleaned data

# Query all beauty purchase data

In [12]:
query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE has_thank_you_confirmation=1 and user_id>50001 AND created_date BETWEEN DATE('2016/09/01') AND DATE('2017/01/20')"

In [13]:
#query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE has_thank_you_confirmation = 1 AND is_clinique=1 AND is_clinique_exposed=1 AND created_date BETWEEN DATE('2016/09/25') AND DATE('2016/12/31')"

In [14]:
#query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE url LIKE '%urban%decay%' and title like '%checkout%' AND user_id>50001"

In [15]:
cursor.execute(query)
#AND user_id > 50001

In [16]:
data = pd.DataFrame(cursor.fetchall(),columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date'])

In [17]:
data

,id,user_id,url,title,domain,created_date
0,1517,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:14:29
1,1522,50008,https://www.urbanoutfitters.com/urban/user/che...,Login - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:14:43
2,1523,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:14:46
3,1544,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:15:56
4,1566,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:16:46
5,1579,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:17:39
6,1639,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:21:17
7,1650,50008,https://www.urbanoutfitters.com/urban/checkout...,Secure Checkout - Urban Outfitters,www.urbanoutfitters.com,2016-09-12 18:21:43
8,4606,50154,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-09-12 20:13:07
9,4904,50154,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-09-12 20:22:25


In [21]:
# this function takes a series of PURCHASE TIMES and returns the indices of urls that are no less than delta 
# minutes apart

def clean_checkout_repeats(userdata, delta):
    cleaned_indices = []
    user_index = list(userdata.index)
    for count, index in enumerate(user_index):
        if user_index[count] > -1 and user_index[count] < max(user_index):
            if (userdata['created_date'][user_index[count]] + datetime.timedelta(minutes = delta) < userdata['created_date'][user_index[count + 1]])\
            or (userdata['domain'][user_index[count]] != userdata['domain'][user_index[count+1]]):
                cleaned_indices.append(user_index[count])
                
    return userdata.loc[cleaned_indices,:]
    #return indices where row + 1 and -1 > delta

In [22]:
cleaned_data = pd.DataFrame(columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date']) 
     
for user in data['user_id'].unique():
    cleaned_data = pd.concat((cleaned_data,clean_checkout_repeats(data.loc[data['user_id']==user], 60)))
    

In [23]:
print(len(data['user_id'].unique()))
#cleaned_data=data.loc[[0,1,2,11],:]
print(cleaned_data)
#print(data)


600
               id  user_id                                                url  \
7          1650.0  50008.0  https://www.urbanoutfitters.com/urban/checkout...   
22         7901.0  50008.0  https://www.facebook.com/v2.2/dialog/oauth?cli...   
104       85755.0  50008.0  https://www.kyliecosmetics.com/11335964/checko...   
120       94851.0  50008.0  https://www.amazon.com/gp/buy/thankyou/handler...   
142      167969.0  50008.0  https://www.amazon.com/gp/buy/thankyou/handler...   
282      222612.0  50008.0                     https://www.hm.com/us/checkout   
285      226117.0  50008.0  https://www.amazon.com/gp/buy/addressselect/ha...   
323      248016.0  50008.0  https://www.amazon.com/gp/buy/thankyou/handler...   
1313     798546.0  50008.0  https://www.freshdirect.com/expressco/checkout...   
1375     819778.0  50008.0  https://www.urbanoutfitters.com/urban/checkout...   
1727     924221.0  50008.0  https://www.urbanoutfitters.com/urban/checkout...   
1833     963038.0  50008

# Function to determine average time between domain and purchase

# version 2

In [ ]:
tic = time.time()

list_of_content_domains = ['sheknows.com']
number_of_days_prior = 1
purchase_qualifier = 'AND domain LIKE "%amazon.com%'
list_of_purchases = cleaned_data

contentdomains = {x:None for x in list_of_content_domains}


user_ids = list_of_purchases['user_id'].unique()  

for domain in list_of_content_domains:

    grand_mean = datetime.timedelta(seconds=0)
    user_counter = 0
        
    for user in user_ids:

        time_domain=dict()
        fre_domain=dict()
        user_times = {x:None for x in contentdomains.keys()}
        user_purchases = list_of_purchases.loc[list_of_purchases['user_id']==user]

        for created_date in user_purchases['created_date']:
                    
            last_x_query = '''SELECT id, user_id, url, created_date, title FROM e_url_map WHERE user_id = {0}\
            AND created_date BETWEEN '{1}' - INTERVAL 7 DAY AND '{1}' \
            AND domain LIKE '%{2}%'  ORDER BY created_date DESC  '''\
            .format(user,created_date,domain)
            cursor.execute(last_x_query)
            last_x_days = pd.DataFrame(cursor.fetchall(), columns = ('id', 'user_id', 'url', 'created_date', 'title'))
            if len(last_x_days)>0:
                
                TIMEDI=created_date - last_x_days['created_date'][0]
                
                if domain in time_domain:
                    time_domain[domain]=time_domain[domain]+TIMEDI
                    fre_domain[domain]=fre_domain[domain]+1
                else:                   
                    time_domain[domain]=TIMEDI
                    fre_domain[domain]=1
            else:
                pass
        try:
            user_ave=time_domain[domain]/fre_domain[domain]
            grand_mean = grand_mean + user_ave
            user_counter += 1
        except:
            pass
    grand_mean = grand_mean/user_counter
    print(domain, grand_mean, user_counter)
    
    toc = time.time()-tic
    print(toc)

# purchasing time

query all browsing data

In [4]:
query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE is_beauty=1 AND user_id > 50001"

In [5]:
cursor.execute(query)

In [6]:
data = pd.DataFrame(cursor.fetchall(),columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date'])

# before purchasing browsing history

In [11]:
cleaned_data.iloc[1778,:]

id                                                    1.41590e+07
user_id                                                     51139
url             https://www.macys.com/service/order-status?cm_...
title                    Check My Order Status & History - Macy's
domain                                              www.macys.com
created_date                                  2016-12-30 23:12:11
Name: 16464, dtype: object

In [24]:
cleaned_data.reset_index()

,index,id,user_id,url,title,domain,created_date
0,5,5074.0,50154.0,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-09-12 20:29:33
1,13,13821.0,50154.0,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-09-13 14:48:12
2,214,508055.0,50154.0,https://www.amazon.com/gp/buy/thankyou/handler...,Amazon.com Thanks You,www.amazon.com,2016-09-29 17:03:01
3,552,774630.0,50154.0,https://www.ebates.com/gilt-groupe_11083-xfas?...,Ebates,www.ebates.com,2016-10-04 20:40:53
4,649,827838.0,50154.0,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-10-05 15:48:59
5,1046,1129389.0,50154.0,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-10-09 17:09:20
6,1114,1200987.0,50154.0,https://www.amazon.com/ap/signin?_encoding=UTF...,Amazon Sign In,www.amazon.com,2016-10-10 17:37:46
7,1441,1517898.0,50154.0,https://www.sephora.com/checkout/shipaddress,Sephora Checkout,www.sephora.com,2016-10-13 17:08:24
8,1639,1740974.0,50154.0,https://www.amazon.com/gp/buy/thankyou/handler...,Amazon.com Thanks You,www.amazon.com,2016-10-15 14:32:50
9,3048,3273316.0,50154.0,https://www.amazon.com/ap/signin?_encoding=UTF...,Amazon Sign In,www.amazon.com,2016-10-23 17:12:05


# Journey(get journey 7 days before purchases)(Clinique)

In [27]:
#AND (url like '%urban%decay%' OR (url like ('%beauty%' OR '%skincare%' OR '%makeup%')))
import re   

list_of_purchases = cleaned_data.reset_index()
fw=open('3.16-wnw-Journey.csv','w')

N=0
#len(cleaned_data)
brand_purchases_index=[]

for i in range (0,len(cleaned_data)):
    print(i)
    fre_domain[i]=dict()
    
    user_ids = list_of_purchases['user_id'][i]
    date=list_of_purchases['created_date'][i]
    #For all domains
    #AND (url like '%clinique%' OR (is_publisher=1 and is_beauty=1))
    
    #only publisher domain
    
    
    last_x_query_brand = '''SELECT id, user_id, url, date(created_date), title,domain,is_publisher,is_beauty FROM e_url_map WHERE user_id = {0}\
   
    AND created_date BETWEEN '{1}' - INTERVAL 7 DAY AND '{1}' \
    
    AND (url like '%wet%wild%' OR (is_publisher=1 and is_beauty=1))
    
    ORDER BY created_date DESC  '''\
    .format(user_ids,date)
    cursor.execute(last_x_query_brand)
    last_x_days_brand = pd.DataFrame(cursor.fetchall(), columns = ('id', 'user_id', 'url', 'created_date', 'title','domain',',is_publisher','is_beauty'))
    
    
    last_x_query = '''SELECT id, user_id, url, created_date, title,domain,is_publisher,is_beauty FROM e_url_map WHERE user_id = {0}\
   
    AND created_date BETWEEN '{1}' - INTERVAL 7 DAY AND '{1}' \
    
    ORDER BY created_date DESC  '''\
    .format(user_ids,date)
    cursor.execute(last_x_query)
    last_x_days = pd.DataFrame(cursor.fetchall(), columns = ('id', 'user_id', 'url', 'created_date', 'title','domain',',is_publisher','is_beauty'))
    
    for url in last_x_days['url'][0:9]:
#change the beauty brand name here c
##if url.find('clinique')!=-1:
###if re.match('(.*?)estee(.*?)lauder(.*?)', url):
        url=url.lower()
        if re.match('(.*?)wet(.*?)wild(.*?)', url):
            print(i)
            brand_purchases_index.append(i)
            fw.write('index='+str(N)+','+'user_id='+str(user_ids)+','+'Time Difference'+','+'Weight'+'\n')

            for x in range (0,len(last_x_days_brand)):
                d1 = last_x_days_brand['created_date'][0]
                d2 = last_x_days_brand['created_date'][x]
                Weight=7-(d1-d2).days
                
                fw.write(last_x_days_brand['url'][x]+','+str(last_x_days_brand['created_date'][x])+','+str((d1-d2).days)+','+str(Weight)+'\n')
            
#            for url in last_x_days_brand['url']:
                
#                fw.write(url+'\t')
#            for created_date in last_x_days_brand['created_date']:
#                fw.write(str(created_date)+'\n')
            for domain in last_x_days_brand['domain']:
                if domain in fre_domain[i]:
                    fre_domain[i][domain]=fre_domain[i][domain]+1
                else:                   
                    fre_domain[i][domain]=1  
            N+=1
            break


print(N)
fw.close()    
#    for url in last_x_days['url']:
#        print(url)
#    for created_date in last_x_days['created_date']:
#        print(created_date)
      
#        if domain in fre_domain[i]:
#            fre_domain[i][domain]=fre_domain[i][domain]+1
#        else:                   
#            fre_domain[i][domain]=1

#    print(i) 
#    print(fre_domain[i])
#fw=open('browsing history.txt','w')
#fw.write(i+fre_domain[i]+'\n')
#fw.close()
    
   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


KeyboardInterrupt: 

In [7]:
att

['0']

# Add values into Attribution(search,shopping cart,coupon...)

# save csv file as txt file

In [15]:

import re
f=open('3.17-Revlon-Journey.txt')
w=open('3.17-revlon-JourneyScore.csv','w')

a=0

for line in f:
    list=[]
    line=line.strip()
    att=line.split('\t')
#    print(att[0])
    
#shopping cart
    url=att[0]
    url=url.lower()
    if a==0:
        w.write(str(att[0])+','+str(att[1])+','+str(att[2])+','+str(att[3])+','+'Shopping Cart'+','+'Search'+','+'Shop'+','+'Coupon'+','+'Review'+','+'Publisher'+'\n')
        print(url)
    else:
        if re.match('(.*?)check(.*?)out(.*?)',url):
            list.append(1)
        elif re.match('(.*?)basket(.*?)',url):
            list.append(1)
        elif re.match('(.*?)cart(.*?)',url):
            list.append(1)
        else:
            list.append(0)
    

#SEARCH   
    
#for url in att[0]:
#    print(url)
        if re.match('(.*?)google(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)yahoo(.*?)revlon(.*?)',url):
            list.append(1)  
        else:
            list.append(0)
    
            

#shop     
#for url in att:
#    print(url)
        if re.match('(.*?)sephora(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)revlon.com(.*?)product(.*?)',url):
            list.append(1)
        elif re.match('(.*?)macys(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)ulta(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)nordstrom(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)tjmaxx(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)shop(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)hsn(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)amazon(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)ebay(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)jcpenney(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)neimanmarcus(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)bergdorfgoodman(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)drugstore(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)saksfifthavenue(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)bloomingdales.com(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)target.com(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)fragrancenet.com(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)bonanza.com(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)overstock.com(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)shopyourway.com(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)cvs.com(.*?)revlon(.*?)',url):
            list.append(1)
        elif re.match('(.*?)belk.com(.*?)revlon(.*?)',url):
            list.append(1)
        else:
            list.append(0)
    
#coupon  
#giltcity.com
#ebates.com  likeacoupon.com
#for url in att:
#    print(url)
        if re.match('(.*?)giltcity.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)ebates.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)groupon.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)likeacoupon.com(.*?)',url):
            list.append(1)
        else:
            list.append(0)


#review
#makeupalley.com
#thenotice.net makeupalley.com
#for url in att[0]:
#    print(url)
        if re.match('(.*?)makeupalley.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)thenotice.net(.*?)',url):
            list.append(1)
        elif re.match('(.*?)nouveaucheap.blogspot.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)phyrra.net(.*?)',url):
            list.append(1)
        elif re.match('(.*?)musingsofamuse.com(.*?)',url):
            list.append(1)
        else:
            list.append(0)
     

#publisher        
#for url in att:
#    print(url)
        if re.match('(.*?)refinery29(.*?)',url):
            list.append(1)
        elif re.match('(.*?)byrdie.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)buzzfeed.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)byrdie.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)allure.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)elle.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)nouveaucheap.blogspot.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)marieclaire.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)glamour.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)purewow.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)rstyle.me(.*?)',url):
            list.append(1)
        elif re.match('(.*?)cosmopolitan.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)temptalia.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)womansday.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)naturallymeblog.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)popsugar.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)extratv.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)acne.org(.*?)',url):
            list.append(1)
        elif re.match('(.*?)seventeen.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)promotions.people.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)thezoereport.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)instyle.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)whowhatwear.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)goodhousekeeping.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)harpersbazaar.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)hellogiggles.com(.*?)',url):
            list.append(1)      
        elif re.match('(.*?)musingsofamuse.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)usmagazine.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)shefinds.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)nymag.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)people.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)huffingtonpost.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)sheknows.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)inthefrow.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)bustle.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)frmheadtotoe.com(.*?)',url):
            list.append(1)
        elif re.match('(.*?)marieclaire.com(.*?)',url):
            list.append(1)
        else:
            list.append(0)
        #print(list)
        w.write(url+','+str(att[1])+','+str(att[2])+','+str(att[3])+','+str(list[0])+','+str(list[1])+','+str(list[2])+','+str(list[3])+','+str(list[4])+','+str(list[5])+'\n')
    a+=1
f.close()
w.close()

0


# Calculate Scores

In [ ]:
import re
import pandas as pd
f=pd.read_csv('ATTRIBUTION.csv')


a=0

for line in f:
    list=[]
    line=line.strip()
    att=line.split('\t')
#    print(att[0])
    
#shopping cart
    url=att[0]
    url=url.lower()
    if a==0:
        w.write(str(att[0])+','+str(att[1])+','+str(att[2])+','+str(att[3])+','+'Shopping Cart'+','+'Search'+','+'Shop'+','+'Coupon'+','+'Review'+','+'Publisher'+'\n')
#    print(url)
    else:
        if re.match('(.*?)check(.*?)out(.*?)',url):
            list.append(1)
        elif re.match('(.*?)basket(.*?)',url):
            list.append(1)
        elif re.match('(.*?)cart(.*?)',url):
            list.append(1)
        else:
            list.append(0)
    

#SEARCH   

In [61]:
brand_purchases_index

[0, 1, 2, 3]

In [62]:
for i in brand_purchases_index:
    print(fre_domain[i])

{'www.clinique.com': 4, 'www.google.com': 2, 'www.allure.com': 1}
{'www.giltcity.com': 3, 'www.google.com': 88, 'thenotice.net': 8, 'www.musingsofamuse.com': 14, 'www.wiseshe.com': 2, 'www.birchbox.com': 6, 'thezoereport.com': 3, 'www1.macys.com': 9, 'shop.nordstrom.com': 6, 'www.lordandtaylor.com': 2, 'karlasugar.net': 3, 'www.dutchessroz.com': 1, 'www.pinterest.com': 2, 'www.reddit.com': 3, 'www.makeupandbeautyblog.com': 22, 'www.clinique.com': 21, 'www.sephora.com': 19, 'beautybanter.com': 3, 'rocaille-writes.blogspot.com': 2, 'theradiancereport.com': 3, 'www.clinique.co.uk': 1, 'theshethings.com': 1, 'www.mywomenstuff.com': 1, 'www.everything-thatmatters.com': 2, 'thandiekay.com': 1, 'www.laurenlovesblog.co.uk': 2, 'www.makeupalley.com': 3, 'www.paulaschoice.com': 5, 'www.perilouslypale.com': 2, 'www.temptalia.com': 2, 'www1.bloomingdales.com': 8, 'thebeautyjunkee.blogspot.com': 1, 'clinique-bonus.com': 9, 'www.saksfifthavenue.com': 1}
{'www.clinique.com': 9, 'www.ulta.com': 4, 'ww

# Calculate Frequency

In [63]:
for i in brand_purchases_index:
    print(i)
    for domain in fre_domain[i]:
        print(domain,fre_domain[i][domain])

0
www.clinique.com 4
www.google.com 2
www.allure.com 1
1
www.giltcity.com 3
www.google.com 88
thenotice.net 8
www.musingsofamuse.com 14
www.wiseshe.com 2
www.birchbox.com 6
thezoereport.com 3
www1.macys.com 9
shop.nordstrom.com 6
www.lordandtaylor.com 2
karlasugar.net 3
www.dutchessroz.com 1
www.pinterest.com 2
www.reddit.com 3
www.makeupandbeautyblog.com 22
www.clinique.com 21
www.sephora.com 19
beautybanter.com 3
rocaille-writes.blogspot.com 2
theradiancereport.com 3
www.clinique.co.uk 1
theshethings.com 1
www.mywomenstuff.com 1
www.everything-thatmatters.com 2
thandiekay.com 1
www.laurenlovesblog.co.uk 2
www.makeupalley.com 3
www.paulaschoice.com 5
www.perilouslypale.com 2
www.temptalia.com 2
www1.bloomingdales.com 8
thebeautyjunkee.blogspot.com 1
clinique-bonus.com 9
www.saksfifthavenue.com 1
2
www.clinique.com 9
www.ulta.com 4
www.google.com 2
thezoereport.com 5
3
www.giltcity.com 3


# Each domain - frequency(sum)---ALL PURCHASES

In [64]:
#len(cleaned_data)
for i in brand_purchases_index:
    print(i)
    for domain in fre_domain[i]:
        if domain in fre_domain[0]:
            fre_domain[0][domain]=fre_domain[0][domain]+fre_domain[i][domain]
        else:                   
            fre_domain[0][domain]=fre_domain[i][domain]
print(fre_domain[0])

0
1
2
3
{'www.clinique.com': 38, 'www.google.com': 94, 'www.allure.com': 2, 'www.giltcity.com': 6, 'thenotice.net': 8, 'www.musingsofamuse.com': 14, 'www.wiseshe.com': 2, 'www.birchbox.com': 6, 'thezoereport.com': 8, 'www1.macys.com': 9, 'shop.nordstrom.com': 6, 'www.lordandtaylor.com': 2, 'karlasugar.net': 3, 'www.dutchessroz.com': 1, 'www.pinterest.com': 2, 'www.reddit.com': 3, 'www.makeupandbeautyblog.com': 22, 'www.sephora.com': 19, 'beautybanter.com': 3, 'rocaille-writes.blogspot.com': 2, 'theradiancereport.com': 3, 'www.clinique.co.uk': 1, 'theshethings.com': 1, 'www.mywomenstuff.com': 1, 'www.everything-thatmatters.com': 2, 'thandiekay.com': 1, 'www.laurenlovesblog.co.uk': 2, 'www.makeupalley.com': 3, 'www.paulaschoice.com': 5, 'www.perilouslypale.com': 2, 'www.temptalia.com': 2, 'www1.bloomingdales.com': 8, 'thebeautyjunkee.blogspot.com': 1, 'clinique-bonus.com': 9, 'www.saksfifthavenue.com': 1, 'www.ulta.com': 4}


# Write avg. frequency in a file

In [65]:
fw=open('3.7-90clinique-domain+avg.frequency.csv','w')
for domain in fre_domain[0]:
        fw.write(domain+','+str(fre_domain[0][domain])+','+str(fre_domain[0][domain]/len(brand_purchases_index))+'\n')
fw.close()

In [28]:
fw=open('urbandecay-journey.csv','w')
for i in range (0,len(cleaned_data)):
    print(i)
    for url in last_x_days['url']:
        fw.write(url+'\n')
    for created_date in last_x_days['created_date']:
        fw.write(str(created_date)+'\n')
fw.close()

0
1
2
3
4
5
6
7
8
